In [ ]:
!nvidia-smi

Fri Dec  9 23:14:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   73C    P8    11W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers
!pip install datasets
!pip install sacrebleu

In [ ]:
from transformers import AutoTokenizer
from datasets import load_dataset, load_metric

In [ ]:
!pip install evaluate
!pip install bert_score
!pip install -U nltk

In [ ]:
# raw_datasets = load_dataset("wmt16", "de-en")

from evaluate import load

bleu_metric = load_metric("sacrebleu")
meteor_metric = load("meteor")
bert_metric = load("bertscore")

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  """
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [ ]:
raw_dataset_train = load_dataset("wmt16", "de-en", split="train[:20%]")
raw_dataset_test = load_dataset("wmt16", "de-en", split="test[:20%]")
raw_dataset_validation = load_dataset("wmt16", "de-en", split="validation[:20%]")




Found cached dataset wmt16 (/root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227)
Found cached dataset wmt16 (/root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227)
Found cached dataset wmt16 (/root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227)


In [ ]:
from datasets import DatasetDict 
raw_datasets = DatasetDict({'train': raw_dataset_train, 'test': raw_dataset_test, 'validation': raw_dataset_validation})

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("t5-small")


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5_fast.py:165: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-small automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  FutureWarning,


In [ ]:
prefix = "translate English to German: "

In [ ]:
max_input_length = 128
max_target_length = 128
source_lang = "en"
target_lang = "de"

def preprocess_function(examples):
    inputs = [prefix + ex[source_lang] for ex in examples["translation"]]
    targets = [ex[target_lang] for ex in examples["translation"]]
    model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)

    # Setup the tokenizer for targets
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=max_target_length, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [ ]:
tokenized_datasets = raw_datasets.map(preprocess_function, batched=True)

  0%|          | 0/910 [00:00<?, ?ba/s]

/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:3579: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  "`as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your "
Loading cached processed dataset at /root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227/cache-f3adb77f5efb5f97.arrow
Loading cached processed dataset at /root/.cache/huggingface/datasets/wmt16/de-en/1.0.0/746749a11d25c02058042da7502d973ff410e73457f3d305fc1177dc0e8c4227/cache-48ae8fac33f43709.arrow


In [ ]:
from tqdm import tqdm

In [ ]:
tokenized_datasets.save_to_disk('./tokenizer')

In [ ]:
from datasets import load_from_disk
tokenized_datasets = load_from_disk('./tokenizer')

In [ ]:
from transformers import AutoModelForSeq2SeqLM

model = AutoModelForSeq2SeqLM.from_pretrained("t5-small")
model.cuda()

T5ForConditionalGeneration(
  (shared): Embedding(32128, 512)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 512)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=512, out_features=512, bias=False)
              (k): Linear(in_features=512, out_features=512, bias=False)
              (v): Linear(in_features=512, out_features=512, bias=False)
              (o): Linear(in_features=512, out_features=512, bias=False)
              (relative_attention_bias): Embedding(32, 8)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=512, out_features=2048, bias=False)
              (wo): Linear(in_features=2048, out_features=512, bias=False)
              (dropout): Drop

In [ ]:
i = 0
for name, param in model.named_parameters():
  if i == 0:
    i += 1
    continue
  split_name = name.split(".")
  if split_name[2] == '0' or split_name[2] == '1' or split_name[2] == '2':
    param.requires_grad = False

In [ ]:
from transformers import DataCollatorForSeq2Seq

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

In [ ]:
!pip install wandb

In [ ]:
!wandb login
%env WANDB_PROJECT=rl4nmt
%env WANDB_ENTITY=rl4nmt

wandb: Currently logged in as: parthsr. Use `wandb login --relogin` to force relogin
env: WANDB_PROJECT=rl4nmt
env: WANDB_ENTITY=rl4nmt


In [ ]:
def compute_metrics(pred):
  labels_ids = pred.label_ids
  pred_ids = pred.predictions[0]

  pred_ids[pred_ids == -100] = tokenizer.pad_token_id
  pred_str = tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
  labels_ids[labels_ids == -100] = tokenizer.pad_token_id
  label_str = tokenizer.batch_decode(labels_ids, skip_special_tokens=True)

  bert_score = 0
  bert_results = bert_metric.compute(predictions=pred_str, references=label_str, model_type="distilbert-base-uncased")
  bert_score += bert_results['f1'][0]

  label_str_list = [[i] for i in label_str]
  bleu_score = 0
  bleu_results = bleu_metric.compute(predictions=pred_str, references=label_str_list)
  bleu_score += bleu_results['score']

  meteor_score = 0
  meteor_results = meteor_metric.compute(predictions=pred_str, references=label_str_list)
  meteor_score += meteor_results['meteor']

  out_dict = {'bert_score': bert_score, 'bleu_score': bleu_score, 'meteor_score': meteor_score}

  return out_dict

In [ ]:
import torch
def preprocess_logits_for_metrics(logits, labels):
    """
    Original Trainer may have a memory leak. 
    This is a workaround to avoid storing too many tensors that are not needed.
    """
    pred_ids = torch.argmax(logits[0], dim=-1)
    return pred_ids, labels

In [ ]:
metric = bleu_metric

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from torch import nn
from torch.distributions import Categorical
import sys, torch

class CustomTrainer(Seq2SeqTrainer):
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.bleu = []
        self.distributions = []
        self.counter = 0 

    def compute_loss(self, model, inputs, return_outputs=False):
        log_probs = []
        labels = inputs.get("labels")
        # forward pass
        outputs = model(**inputs)
        logits = outputs.get("logits")
        attn_mask = torch.where(labels != -100, 1, 0)
        masked_labels = labels*attn_mask
        decoded_labels = tokenizer.batch_decode(masked_labels,skip_special_tokens=True)
        decoded_labels = [[i] for i in decoded_labels]
        
        attn_mask = torch.unsqueeze(attn_mask, 2)
        masked_logits = logits*attn_mask
        decoded_logits = []
        log_probs_batch = []
        cat = Categorical(logits=masked_logits)
        samplebatch = cat.sample()
        log_probs_batch = cat.log_prob(samplebatch)
        log_probs_batch = torch.mean(log_probs_batch,1)
        decoded_logits = tokenizer.batch_decode(samplebatch, skip_special_tokens=True)
        bleu_scores = [metric.compute(predictions=[decoded_logits[i]], references=[decoded_labels[i]])["score"] for i in range(len(decoded_labels))]
        loss = sum([-log_prob*bleu_score for log_prob, bleu_score in zip(log_probs_batch, bleu_scores)])
        return (loss, outputs) if return_outputs else loss

In [ ]:
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer

training_args = Seq2SeqTrainingArguments(
    output_dir="./results",
    evaluation_strategy="steps",
    eval_steps=25000,
    save_strategy="epoch", 
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    weight_decay=0.01,
    save_total_limit=10,
    num_train_epochs=15,
    fp16=True,
    report_to="wandb",
    run_name="T5-Small-BLEURL-Finetuning-20.1"
)

trainer = CustomTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    preprocess_logits_for_metrics=preprocess_logits_for_metrics
)

Using cuda_amp half precision backend


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation. If translation are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:310: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,
***** Running training *****
  Num examples = 909777
  Num Epochs = 15
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 852930
  Number of trainable parameters = 38478336
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of th

You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss,Bert Score,Bleu Score,Meteor Score
25000,757.870500,803.000000,0.791669,12.177567,0.459398
50000,432.310500,640.500000,0.733356,10.108712,0.409512
75000,384.557300,575.000000,0.790668,8.930599,0.381029
100000,329.544000,528.000000,0.790277,7.594613,0.349542
125000,280.246000,461.250000,0.756751,7.183621,0.323491
150000,260.818800,409.250000,0.706570,6.962001,0.306482
175000,250.816200,369.750000,0.828171,7.073449,0.291731
200000,254.613800,378.750000,0.792259,6.183392,0.279009
225000,256.019200,343.000000,0.706556,6.125310,0.265195
250000,258.934400,332.000000,0.832386,6.032403,0.265420


The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation. If translation are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 434
  Batch size = 16


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_tok

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased/snapshots/1c4513b2eedbda136f57676a34eea67aba266e5c/pytorch_model.bin
All the weights of DistilBertModel were initialized from the model checkpoint at distilbert-base-uncased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertModel for predictions without further training.
The following columns in the evaluation set don't have a corresponding argument in `T5ForConditionalGeneration.forward` and have been ignored: translation. If translation are not expected by `T5ForConditionalGeneration.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 434
  Batch size = 16
Saving model checkpoint to ./results/checkpoint-56862
Configuration saved in ./results/checkpoint-56862/config.json
Model weights saved in ./results/checkpoint-56862/pytorch_model.bin
tokenizer config file saved in ./

TrainOutput(global_step=852930, training_loss=288.87533790690617, metrics={'train_runtime': 147667.8159, 'train_samples_per_second': 92.415, 'train_steps_per_second': 5.776, 'total_flos': 2.862887666839388e+17, 'train_loss': 288.87533790690617, 'epoch': 15.0})

In [ ]:
from transformers import pipeline

model_checkpoint = "results/checkpoint-284000"
translator = pipeline("translation_en_to_de", model=model_checkpoint)
print(translator("How are you?"))

input_ids = tokenizer("translate English to German: how are you", return_tensors="pt").input_ids.to('cuda')
outputs = model.generate(input_ids)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
!apt install git-lfs

In [ ]:
model.push_to_hub("t5-11785")